In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [16]:
import json
import pandas as pd
import numpy as np
import glob 
from tabulate import tabulate 

pd.set_option('display.max_colwidth', None)

In [3]:
import torch 
import random
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)

In [4]:
mace_annotation_df = pd.read_pickle('./data/annotation-results/MACE-measure/final_mace_predictions.pkl')

In [5]:
mace_annotation_df['turn_text_with_topic'] = mace_annotation_df.apply(lambda row: {
                                                                        'author': row['turn_text']['author'], 
                                                                        'text' : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                                                       } ,axis=1)

In [6]:
mace_annotation_df.head()

,task_id,turn_id,topic_func_label,dlg_act_label,exp_act_label,topic,lvl,turn_text,turn_text_with_topic
0,142,1,(T04) Other - No topic was introduced,(D09) To provide informing statement,(E02) Testing prior knowledge,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'Mostly today, I've been talking a lot about what can we do, what's possible. We think might be possible in the next couple of years. But really at the professional level its more the question of wisdom of what should we be doing.'}","{'author': 'Explainer', 'text': 'virtual reality [SEP] Mostly today, I've been talking a lot about what can we do, what's possible. We think might be possible in the next couple of years. But really at the professional level its more the question of wisdom of what should we be doing.'}"
1,142,2,(T01) It is the main topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'That's one of the things we're trying to figure out, from an artist and storytelling perspective, what are the things that will make this meaningfully different from what we're used to, like a television on our wall. And we've been finding a lot of things, aspects of virtual reality that very much do that in my opinion. Things that allow you to feel presence, first and foremost, where you get lost, and you have to remind yourself, this isn't actually happening. And things that ultimately allow you to embody other characters. Things where you can actually change your own self-perception and play with neuro-plasticity and teach yourself things that are bizarre and unique.'}","{'author': 'Explainee', 'text': 'virtual reality [SEP] That's one of the things we're trying to figure out, from an artist and storytelling perspective, what are the things that will make this meaningfully different from what we're used to, like a television on our wall. And we've been finding a lot of things, aspects of virtual reality that very much do that in my opinion. Things that allow you to feel presence, first and foremost, where you get lost, and you have to remind yourself, this isn't actually happening. And things that ultimately allow you to embody other characters. Things where you can actually change your own self-perception and play with neuro-plasticity and teach yourself things that are bizarre and unique.'}"
2,142,3,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'As an engineer of course, I love quantifiable things. I like saying here's my 18 millisecond motion to photon, here's my angular resolution that I'm improving. I'm doing the color-space right. But you can look not too far back where you say we have blu-ray DVDs at this amazing resolution, but more people want to watch Youtube videos at really bad early internet video speeds. Where there are things that if you deliver a value to people then these objective quantities may not be the most important thing. And while we're certainly pushing as hard as we can on lots of these things that make the experience better in potentially every way or maybe just for videos or for different things. I don't think that its necessary. I've commented that I think usually my favorite titles on mobile that are fully synthetic are ones that don't even try, they just go and do light-mapped, black-shaded. And I think its a lovely aesthetic. I think that you don't wind up fighting all of the aliasing. While you get some other titles that, oh we're gonna be high-tech with our specular bump maps with roughness. And you've got aliasing everywhere, and you can't hold frame rate and its all problematic. While some of these that are clearly very synthetic worlds where its nothing but these cartoony flat-shaded things with lighting, but they look and they feel good. And you can buy that you're in that place. And you want to know what's around that monolith over there.'}","{'

In [7]:
mace_annotation_df.topic_func_label.value_counts()

(T04) Other - No topic was introduced    772
(T01) It is the main topic               430
(T03) A related topic                    260
(T02) A subtopic                          88
Name: topic_func_label, dtype: int64

In [8]:
mace_annotation_df.dlg_act_label.value_counts()

(D09) To provide informing statement           696
(D07) To provide agreement statement           265
(D01) To ask a check question                  245
(D02) To ask what/how question                 115
(D10) Other                                    101
(D04) To answer a question by confirming        54
(D06) To answer - Other                         25
(D05) To answer a question by disconfirming     24
(D03) To ask other kind of questions            13
(D08) To provide disagreement statement         12
Name: dlg_act_label, dtype: int64

In [9]:
mace_annotation_df.exp_act_label.value_counts()

(E03) Provide an explanation                679
(E07) Providing Feedback                    285
(E04) Ask for an explanation                142
(E05) Signaling understanding               141
(E02) Testing prior knowledge               112
(E10) Other                                  59
(E01) Testing understanding                  56
(E09) Introducing Extraneous Information     48
(E06) Signaling non-understanding            17
(E08) Providing Assessment                   11
Name: exp_act_label, dtype: int64

### Models to experiment with:
- Majority label baseline
- Basic BERT model
- A model that expose the dependency between the label
- A model that takes into consideration labels of other sequences in the dialog.

### How to evaluate:
- Cross domain evaluation: for each topic train the model on all other topics and evaluate on the selected topic (13-fold cross validation)

### Majority class:

In [26]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [27]:
def majority_class(df):
    topics = df.topic.unique()
    for topic in topics:
        training_df = df[df.topic != topic]
        #compute the majority class for each label
        l = len(df[df.topic == topic])
        df.loc[df.topic == topic, 'topic_func_maj_pred'] = [training_df.topic_func_label.mode()] * l
        df.loc[df.topic == topic, 'dlg_act_maj_pred']    = [training_df.dlg_act_label.mode()] * l
        df.loc[df.topic == topic, 'exp_act_maj_pred']    = [training_df.exp_act_label.mode()] * l
    
    return df

def eval_preds(df, models_names, gt_clms, pred_clms):
    results_table = []
    for label in zip(gt_clms, pred_clms, models_names):
        ground_truths = df[label[0]].tolist()
        predictions   = df[label[1]].tolist()
        model_name = label[2]
        
        class_names = df[label[0]].unique()

        prc_scores = precision_score(ground_truths, predictions, average=None, labels=class_names)
        rec_scores = recall_score(ground_truths, predictions, average=None, labels=class_names)
        f1_scores  = f1_score(ground_truths, predictions, average=None, labels=class_names)
        
        macro_prc_scores = precision_score(predictions, ground_truths, average='macro', labels=class_names)
        macro_rec_scores = recall_score(predictions, ground_truths, average='macro', labels=class_names)
        macro_f1 = f1_score(predictions, ground_truths, average='macro', labels=class_names)
        
        scores ={}
        for i, c in enumerate(class_names):
            scores[c] = {'prec': round(prc_scores[i],2), 'recall': round(rec_scores[i],2), 'f1': round(f1_scores[i],2)}
        
        scores['Macro AVG.'] = {'prec': round(macro_prc_scores,2), 'recall': round(macro_rec_scores,2), 'f1': round(macro_f1,2)}
        
        results_table.append([model_name, label[0], scores])
    
    return results_table

#### Basic BERT:

In [17]:
import transformers
import datasets

import torch
import json
from pathlib import Path
from datasets import load_dataset, Dataset, load_metric
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, default_data_collator,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          TrainingArguments, Trainer)
from transformers.modeling_outputs import SequenceClassifierOutput

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from glob import glob

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [18]:
device

device(type='cuda')

In [16]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

In [23]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    f1score = f1_score(predictions, labels, average='macro')
    return {'f1-score': f1score}

def preprocess_function(examples, input_clm='turn_text'):
    # Tokenize the texts
    texts = [x['text'] for x in examples[input_clm]]
    result = tokenizer(texts, truncation=True, padding='max_length')
    
    if 'labels' in examples:
        result['labels'] = examples['labels']
        
    return result

def compute_model_scores(model, eval_dataset, class_names):
    eval_dataset = eval_dataset.map(preprocess_function, batched=True)
    eval_dataset = eval_dataset.remove_columns(['__index_level_0__', 'dlg_act_label', 'exp_act_label', 'lvl', 'task_id', 'topic', 'topic_func_label', 'turn_id', 'turn_text'])

    eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator)
    all_predictions = []
    all_labels = []
    for step, batch in enumerate(eval_dataloader):
        batch = {x[0]: x[1].cuda() for x in batch.items()}
        outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions+=[x.item() for x in predictions]
        all_labels+=[x.item() for x in batch['labels']]

    prc_scores = precision_score(all_predictions, all_labels, average=None, labels=class_names)
    rec_scores = recall_score(all_predictions, all_labels, average=None, labels=class_names)
    f1_scores  = f1_score(all_predictions, all_labels, average=None, labels=class_names)
    
    return prc_scores, rec_scores, f1_scores

def bert_pred_labels(models_path, df, label_clm, input_clm):
    label_dictionary = {int(l[2:4])-1 : l for l in  df[label_clm].unique()}
    
    df[label_clm + '_bert_pred'] = [None] * len(df)
    
    class_names = list(sorted(df[label_clm]))
    
    for model_path in glob(models_path):
        model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
        eval_topic = model_path.split('/')[-1].replace('model_', '')
        print(eval_topic)
        eval_dataset = Dataset.from_pandas(df[df.topic == eval_topic])
        eval_dataset = eval_dataset.map(lambda examples: preprocess_function(examples, input_clm), batched=True)
        eval_dataset = eval_dataset.remove_columns(df.columns.tolist() + ['__index_level_0__'])
        #eval_dataset = eval_dataset.remove_columns(['__index_level_0__', 'dlg_act_label', 'exp_act_label', 'lvl', 'task_id', 'topic', 'topic_func_label', 'turn_id', input_clm])

        eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator)
        all_predictions = []
        for step, batch in enumerate(eval_dataloader):
            batch = {x[0]: x[1].cuda() for x in batch.items()}
            outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1)
            all_predictions+=[x.item() for x in predictions]

        df.loc[df.topic == eval_topic,label_clm + '_bert_pred'] = [label_dictionary[x] for x in all_predictions]
        
    return df

In [20]:
def train_model(output_dir, train_ds, valid_ds, test_ds, num_labels , num_train_epochs=5):
    batch_size = 4

    args = TrainingArguments(
        output_dir= output_dir,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1-score'
    )
    
    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels).to(device)
    
    multi_trainer =Trainer(
        model,
        args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        compute_metrics=lambda x: compute_metrics(x),
        tokenizer=tokenizer
    )
    
    multi_trainer.train()
    
    model.save_pretrained(output_dir)
    eval_results = multi_trainer.evaluate(test_ds)
    
    return model, eval_results

def cross_topic_eval(df, label_clm, input_clm, output_dir, num_train_epochs=5):
    df['labels'] = df[label_clm].apply(lambda x: int(x[2:4])-1) #making labels parasable as integers
    num_labels = df['labels'].nunique()
    
    topics = df.topic.unique() #Train and evaluate one model for each topic...
    all_eval_results = {}
    for topic in topics:
        print(topic)
        training_df= df[df.topic != topic]
        
        #split the training df to further training and validation per topic
        train_topics = training_df.topic.unique()
        train_topics, test_topics = train_test_split(train_topics, test_size=2)
        
        new_training_df = training_df[training_df.topic.isin(train_topics)].copy()
        valid_df    = training_df[training_df.topic.isin(test_topics)].copy()
        test_df     = df[df.topic == topic].copy()
        
        #balance the data
        new_training_df, y = ros.fit_resample(new_training_df, new_training_df['labels'])
        new_training_df['labels'] = y
        
        train_ds = Dataset.from_pandas(new_training_df)
        valid_ds = Dataset.from_pandas(valid_df)
        test_ds  = Dataset.from_pandas(test_df)
        
        train_ds = train_ds.map(lambda examples: tokenizer([x['text'] for x in examples[input_clm]], truncation=True, padding='max_length'), batched=True)
        valid_ds = valid_ds.map(lambda examples: tokenizer([x['text'] for x in examples[input_clm]], truncation=True, padding='max_length'), batched=True)
        test_ds  = test_ds.map(lambda examples: tokenizer([x['text'] for x in examples[input_clm]], truncation=True, padding='max_length'), batched=True)
        
        eval_results = train_model(output_dir + '_{}'.format(topic), train_ds, valid_ds, test_ds, num_labels, num_train_epochs=num_train_epochs)
        
        all_eval_results[topic] = eval_results

    return all_eval_results

### Training:

- For each topic, train a model on the other 12 topics as follows:
  - From the 12 topics select one for validation and 11 for training
  - Train the model for 10 epochs selecting the checkpoint that gives the highest f1-score on the validation set
  

#### Train for the topic_func_label:

In [ ]:
eval_results = cross_topic_eval(mace_annotation_df.copy(), 'topic_func_label', 'turn_text_with_topic', './data/models/topic_func_models/model', num_train_epochs=5)

virtual_reality


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
res_table = []
for topic, res in eval_results.items():
    res_table.append([topic, res[1]['eval_f1-score']])

res_table.append(['Average', np.mean([x[1] for x in res_table])])

print(tabulate(res_table, headers=['topic', 'f1-score']))

#### Train for the dlg_act_label:

In [ ]:
eval_results = cross_topic_eval(mace_annotation_df.copy(), 'dlg_act_label', 'turn_text', './data/models/dlg_act_label/model')

In [ ]:
res_table = []
for topic, res in eval_results.items():
    res_table.append([topic, res[1]['eval_f1-score']])

res_table.append(['Average', np.mean([x[1] for x in res_table])])

print(tabulate(res_table, headers=['topic', 'f1-score']))

#### Train for the exp_act_label:

In [ ]:
eval_results = cross_topic_eval(mace_annotation_df.copy(), 'exp_act_label', 'turn_text', './data/models/exp_act_label/model')

In [ ]:
res_table = []
for topic, res in eval_results.items():
    res_table.append([topic, res[1]['eval_f1-score']])

res_table.append(['Average', np.mean([x[1] for x in res_table])])

print(tabulate(res_table, headers=['topic', 'f1-score']))

### Prediction using the trained models:

Generate predictions using majority class and our trained baseline models:

In [21]:
mace_annotation_df.columns

Index(['task_id', 'turn_id', 'topic_func_label', 'dlg_act_label',
       'exp_act_label', 'topic', 'lvl', 'turn_text', 'turn_text_with_topic',
       'topic_func_label_bert_pred'],
      dtype='object')

In [24]:
mace_annotation_df = bert_pred_labels('./data/models/topic_func_models/*', mace_annotation_df, 'topic_func_label', 'turn_text_with_topic')
mace_annotation_df = bert_pred_labels('./data/models/exp_act_label/*', mace_annotation_df, 'exp_act_label', 'turn_text')
mace_annotation_df = bert_pred_labels('./data/models/dlg_act_label/*', mace_annotation_df, 'dlg_act_label', 'turn_text')

blackhole


  0%|          | 0/1 [00:00<?, ?ba/s]

sleep_scientist


  0%|          | 0/1 [00:00<?, ?ba/s]

blockchain


...

  0%|          | 0/1 [00:00<?, ?ba/s]

lasers


  0%|          | 0/1 [00:00<?, ?ba/s]

connectome


  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
mace_annotation_df = majority_class(mace_annotation_df)

In [1]:
mace_annotation_df.to_pickle('./majority_and_basic_bert_pred.pkl')

NameError: name 'mace_annotation_df' is not defined